<a href="https://colab.research.google.com/github/bartekwaskii/speechAnonymizer/blob/main/anon_eng_speecht5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Environment setup

##Drive mount

In [ ]:
from google.colab import drive
import os
drive.mount('/content/gdrive/', force_remount=True)
os.chdir('/content/gdrive/MyDrive/colab/tm2024/Grupa3/Kuba/dupa')

Mounted at /content/gdrive/


##Python modules

In [ ]:
!pip install datasets
!pip install speechbrain
!pip install git+https://github.com/openai/whisper.git

from transformers import SpeechT5Processor, SpeechT5ForTextToSpeech, SpeechT5HifiGan
from datasets import load_dataset, Audio
import torch
import soundfile as sf
from IPython.display import Audio
from speechbrain.inference.speaker import EncoderClassifier
import torchaudio
import numpy as np
import warnings
import whisper
warnings.filterwarnings("ignore")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 6.9 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 824.8/824.8 kB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.7/117.7 kB 7.8 MB/s eta 0:00:00
   ━

##Pretrained models

### Speech to Text - [Whisper](https://github.com/openai/whisper)

In [ ]:
transcriptor = whisper.load_model("base.en")

100%|███████████████████████████████████████| 139M/139M [00:02<00:00, 57.6MiB/s]


###xvector extractor - [SpeechBrain Speaker Verification with xvector embeddings on Voxceleb](https://huggingface.co/speechbrain/spkrec-xvect-voxceleb)

In [ ]:
classifier = EncoderClassifier.from_hparams(source="speechbrain/spkrec-xvect-voxceleb", savedir="pretrained_models/spkrec-xvect-voxceleb")

hyperparams.yaml:   0%|          | 0.00/2.04k [00:00<?, ?B/s]

embedding_model.ckpt:   0%|          | 0.00/16.9M [00:00<?, ?B/s]

mean_var_norm_emb.ckpt:   0%|          | 0.00/3.20k [00:00<?, ?B/s]

classifier.ckpt:   0%|          | 0.00/15.9M [00:00<?, ?B/s]

label_encoder.txt:   0%|          | 0.00/129k [00:00<?, ?B/s]

### Synthesizer - [SpeechT5](https://huggingface.co/blog/speecht5)

In [ ]:
processor = SpeechT5Processor.from_pretrained("microsoft/speecht5_tts")
model = SpeechT5ForTextToSpeech.from_pretrained("microsoft/speecht5_tts")
vocoder = SpeechT5HifiGan.from_pretrained("microsoft/speecht5_hifigan")

preprocessor_config.json:   0%|          | 0.00/433 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/232 [00:00<?, ?B/s]

spm_char.model:   0%|          | 0.00/238k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/40.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/234 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/2.06k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/585M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/50.7M [00:00<?, ?B/s]

##xvector database - [CMU ARCTIC](https://huggingface.co/datasets/Matthijs/cmu-arctic-xvectors)

In [ ]:
xvector_database = load_dataset("Matthijs/cmu-arctic-xvectors", split="validation")
xvector_database = torch.tensor(xvector_database["xvector"])

README.md:   0%|          | 0.00/1.01k [00:00<?, ?B/s]

cmu-arctic-xvectors.py:   0%|          | 0.00/1.36k [00:00<?, ?B/s]

0000.parquet:   0%|          | 0.00/21.3M [00:00<?, ?B/s]

Generating validation split:   0%|          | 0/7931 [00:00<?, ? examples/s]

#Analysis

## Speech to text

In [ ]:
transcription = transcriptor.transcribe("input_resynteza.wav")
print(transcription["text"])

 How does it feel? How does it feel? To be on your own with no directions home. A complete unknown like a rolling stone.


##xvect_extraction

In [ ]:
signal, fs = torchaudio.load('input_resynteza.wav')
extracted_xvector = classifier.encode_batch(signal[0]).squeeze(0)
print(np.shape(extracted_xvector))

torch.Size([1, 512])


#Anonymization

In [ ]:
def anonymize(xvector):

  #computing euclidian distance from each database speaker
  #to determine the least similar speakers.
  euclid_dist = torch.cdist(xvector_database, xvector,p=2)
  ed_list = np.reshape(euclid_dist.tolist(), (-1))
  least_similar_idx = ed_list.argsort()[-3:][::-1]

  #averaging the least similar speakers to create anonymized
  axv = []

  for x in range(0,3):
    axv.append(xvector_database[least_similar_idx[x]])

  axv = np.mean(axv, axis=0)
  anon_xvector1 = torch.tensor(axv).unsqueeze(0)


  #max distance speaker
  maxid = np.argmax(ed_list)
  anxv3 = xvector_database[maxid]
  anon_xvector2 = torch.tensor(anxv3).unsqueeze(0)


  #determining random speakers to get a randomly distant, yet distant embedding
  randidx = np.random.randint(0, len(xvector_database),1)
  axv_rand = xvector_database[randidx]
  anon_xvector3 = torch.tensor(axv_rand)

  return anon_xvector1, anon_xvector2, anon_xvector3

In [ ]:
anon_xvector1, anon_xvector2, anon_xvector3 = anonymize(xvector=extracted_xvector)

#Synthesis

##Original signal

In [ ]:
Audio(signal, rate=16000)

##Model

###Original speech resynthesis

In [ ]:
inputs = processor(text=transcription["text"], return_tensors="pt")
speech = model.generate_speech(inputs["input_ids"], extracted_xvector, vocoder=vocoder)
Audio(speech, rate=16000)

###Distance anonimization

In [ ]:
inputs = processor(text=transcription["text"], return_tensors="pt")
speech = model.generate_speech(inputs["input_ids"], anon_xvector1, vocoder=vocoder)
Audio(speech, rate=16000)

###Max distance speaker

In [ ]:
inputs = processor(text=transcription["text"], return_tensors="pt")
speech = model.generate_speech(inputs["input_ids"], anon_xvector2, vocoder=vocoder)
Audio(speech, rate=16000)

###Speaker randomization

In [ ]:
inputs = processor(text=transcription["text"], return_tensors="pt")
speech = model.generate_speech(inputs["input_ids"], anon_xvector3, vocoder=vocoder)
Audio(speech, rate=16000)

#Archive

##Archival anonymization methods



###Averaged randomization
Choosing three random speaker embeddings to average them out into a randomly distant anonymous pseudospeaker.

In [ ]:
#determining random speakers to get a randomly distant, yet distant embedding
randidx = np.random.randint(0, len(xvector_database),3)
axv_rand = list(xvector_database[randidx])
axv_rand = np.mean(axv_rand, axis=0)
anon_xvector1 = torch.tensor(axv_rand).unsqueeze(0)

inputs = processor(text=transcription["text"], return_tensors="pt")
speech = model.generate_speech(inputs["input_ids"], anon_xvector1, vocoder=vocoder)
Audio(speech, rate=16000)